<a href="https://colab.research.google.com/github/SU-sumico/dlj/blob/main/3_cnn_cats_vs_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

目的
* ネコとイヌの画像データと畳み込みニューラルネットワーク（CNN）を用いて分類を行う。

準備
* GPUを用いて処理を行うために、上部のメニューバーの「ランタイム」→「ランタイムのタイプを変更」からハードウェアアクセラレータをGPUにしてください。

使用するデータセット
* データセットは、RGBカラーの画像、ネコとイヌのクラスに分類されています。
https://www.kaggle.com/datasets/karakaggle/kaggle-cat-vs-dog-dataset?resource=download

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

img_w, img_h = 150, 150        # 画像をリサイズするときのサイズ
batch_size = 32                # ミニバッチのサイズ

# 訓練データを読み込んで加工するジェネレータを生成
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,         # RGB値を0～1の範囲に変換
    rotation_range=15,         # 15度の範囲でランダムに回転させる
    zoom_range=0.2,            # ランダムに拡大
    horizontal_flip=True       # 水平方向にランダムに反転
    )

# テストデータを読み込むジェネレータを生成
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# 訓練データをミニバッチの数だけ生成
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Deep learning/3data/train',               # 訓練データのフォルダ
    target_size=(img_w, img_h), # 画像をリサイズ
    batch_size=batch_size,      # ミニバッチのサイズ
    class_mode='binary')        # 二値分類なのでbinaryを指定

# テストデータをミニバッチの数だけ生成
validation_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Deep learning/3data/validation',          # テストデータのフォルダ
    target_size=(img_w, img_h), # 画像をリサイズ
    batch_size=batch_size,      # ミニバッチのサイズ
    class_mode='binary')        # 二値分類なのでbinaryを指定

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


ImageDataGenerator: https://keras.io/ja/preprocessing/image/
* リアルタイムにデータ拡張しながらテンソル画像データのバッチを生成します。また、このジェネレータはデータを無限にループするので、無限にバッチを生成します。

ImageDataGeneratorメソッド

flow_from_directory: https://keras.io/ja/preprocessing/image/
* ディレクトリへのパスを受け取り、拡張/正規化したデータのバッチを生成します。

画像のラベルと番号を確認する方法
* ラベルは、flow_from_directoryで指定したディレクトリにある、フォルダの名前が割り当てられます。
* ディレクトリに「dog」と「cat」というフォルダがあれば、ラベルは、[dog, cat]になります。
* そのラベルには番号が割り振られています。
* それを確認するのが、class_indicesの機能です。
* label:番号で保存されています。

In [ ]:
# 正解ラベルを確認
print(train_generator.class_indices)
print(validation_generator.class_indices)

{'cats': 0, 'dogs': 1}
{'cats': 0, 'dogs': 1}


Sequentialモデル: https://keras.io/ja/getting-started/sequential-model-guide/
* Kerasでモデルを作成するにはSequentialモデル https://keras.io/ja/getting-started/sequential-model-guide/) を用いる方法とFunctionalAPI (https://keras.io/ja/getting-started/functional-api-guide/) を用いる2つの方法があります。
* Sequentialモデルでは、はじめにSequentialクラスのインスタンスを作成します。
* そしてaddメソッドを用いてレイヤ(全結合層やCNN層、プーリング層など)を追加していきます。

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras import optimizers

# モデルを構築/Seqentialモデルのインスタンスを作成
model = Sequential()

# （第1層）畳み込み層/addメソッドでレイヤを追加
model.add(
    Conv2D(
        filters=32,                # フィルターの数は32
        kernel_size=(3, 3),        # 3×3のフィルターを使用
        input_shape=(150, 150, 3), # 入力データの形状
        padding='same',            # ゼロパディングを行う
        activation='relu'          # 活性化関数はReLU
        ))
# （第2層）プーリング層
model.add(
    MaxPooling2D(pool_size=(2, 2))
)
# ドロップアウト25％
model.add(Dropout(0.25))

# （第3層）畳み込み層
model.add(
    Conv2D(
        filters=32,            # フィルターの数は32
        kernel_size=(3, 3),    # 3×3のフィルターを使用
        activation='relu'      # 活性化関数はReLU
        ))
# （第4層）プーリング層
model.add(
    MaxPooling2D(pool_size=(2, 2))
)
# ドロップアウト25％
model.add(Dropout(0.25))

# （第5層）畳み込み層
model.add(
    Conv2D(filters=64,         # フィルターの数は64
           kernel_size=(3, 3), # 3×3のフィルターを使用
           activation='relu'   # 活性化関数はReLU
          ))
# （第6層）プーリング層
model.add(
    MaxPooling2D(pool_size=(2, 2)))
# ドロップアウト25％
model.add(Dropout(0.25))

# 出力層への入力を4階テンソルから2階テンソルに変換する
model.add(Flatten())

# （第7層）全結合層
model.add(
    Dense(64,                  # ニューロン数は64
          activation='relu'))  # 活性化関数はReLU
# ドロップアウト50％
model.add(Dropout(0.5))

# （第8層）出力層
model.add(
    Dense(
        1,                     # ニューロン数は1個
        activation='sigmoid')) # 活性化関数はsigmoid

# モデルのコンパイル
model.compile(
    loss='binary_crossentropy', # バイナリ用の交差エントロピー誤差
    metrics=['accuracy'],       # 学習評価として正解率を指定
    # Adamアルゴリズムで最適化
    optimizer=optimizers.Adam(),
)

# モデルのサマリを表示
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 75, 75, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 75, 75, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 73, 73, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 32)        0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 36, 36, 32)        0

Sequentialモデルのメソッド: https://keras.io/ja/models/sequential/

ImageDataGeneratorを使った学習
* ImageDataGeneratorを利用して学習を行うには、model.fit_generator()を使います。
* fit_generator: https://keras.io/ja/models/model/

In [ ]:
epochs = 60             # 学習回数
batch_size = batch_size # 設定済みのミニバッチのサイズ

# 学習を行う
history = model.fit_generator(
    train_generator,    # 訓練データ
    epochs=epochs,      # 学習回数
    verbose=1,          # 学習の進捗状況を出力する
    # テストデータ
    validation_data=validation_generator,
)

<ipython-input-5-fcec4d70183c>:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/60
63/63 [==============================] - 884s 14s/step - loss: 0.7073 - accuracy: 0.4960 - val_loss: 0.6917 - val_accuracy: 0.5713
Epoch 2/60
63/63 [==============================] - 20s 319ms/step - loss: 0.6867 - accuracy: 0.5340 - val_loss: 0.6920 - val_accuracy: 0.5638
Epoch 3/60
63/63 [==============================] - 21s 335ms/step - loss: 0.6917 - accuracy: 0.5250 - val_loss: 0.6926 - val_accuracy: 0.5500
Epoch 4/60
63/63 [==============================] - 21s 328ms/step - loss: 0.6928 - accuracy: 0.5145 - val_loss: 0.6893 - val_accuracy: 0.5750
Epoch 5/60
63/63 [==============================] - 20s 319ms/step - loss: 0.6784 - accuracy: 0.5610 - val_loss: 0.6870 - val_accuracy: 0.5850
Epoch 6/60
63/63 [==============================] - 21s 333ms/step - loss: 0.6780 - accuracy: 0.5715 - val_loss: 0.6654 - val_accuracy: 0.5650
Epoch 7/60
63/63 [==============================] - 20s 311ms/step - loss: 0.6740 - accuracy: 0.5725 - val_loss: 0.6622 - val_accuracy: 0.5863


テストデータによるネコとイヌの識別は、正解率が76.6%。データ拡張しても精度は低い。